In [1]:
import os
import pandas as pd
import numpy as np
import psycopg2
import plotly.graph_objects as go
import plotly.express as px

pd.options.display.max_columns = 99
%load_ext nb_black

<IPython.core.display.Javascript object>

In [2]:
# connect to the database
conn = psycopg2.connect(
    dbname="warby_parker",
    user="postgres",
    password=os.environ.get("DB_PASSWORD"),
    host="localhost",
)
# create a cursor
curr = conn.cursor()

# show query result as pandas dataframe
def show_result(query, connection=conn):
    return pd.read_sql_query(query, connection)

<IPython.core.display.Javascript object>

In [3]:
# show all the tables in the database
q1 = """
SELECT * FROM pg_catalog.pg_tables
WHERE schemaname != 'pg_catalog' AND schemaname != 'information_schema';
"""
show_result(q1)

,schemaname,tablename,tableowner,tablespace,hasindexes,hasrules,hastriggers,rowsecurity
0,public,survey,postgres,None,False,False,False,False
1,public,quiz,postgres,None,False,False,False,False
2,public,home_try_on,postgres,None,False,False,False,False
3,public,purchase,postgres,None,False,False,False,False


<IPython.core.display.Javascript object>

## Survey  Funnel

In [4]:
# first five rows of survey
q2 = """
SELECT * 
FROM survey
LIMIT 5;
"""
show_result(q2)

,question,user_id,response
0,1. What are you looking for?,005e7f99-d48c-4fce-b605-10506c85aaf7,Women's Styles
1,2. What's your fit?,005e7f99-d48c-4fce-b605-10506c85aaf7,Medium
2,3. Which shapes do you like?,00a556ed-f13e-4c67-8704-27e3573684cd,Round
3,4. Which colors do you like?,00a556ed-f13e-4c67-8704-27e3573684cd,Two-Tone
4,1. What are you looking for?,00a556ed-f13e-4c67-8704-27e3573684cd,I'm not sure. Let's skip it.


<IPython.core.display.Javascript object>

In [5]:
# Unique questions
q3 = """
SELECT DISTINCT question
FROM survey
ORDER BY 1;
"""
show_result(q3)

,question
0,1. What are you looking for?
1,2. What's your fit?
2,3. Which shapes do you like?
3,4. Which colors do you like?
4,5. When was your last eye exam?


<IPython.core.display.Javascript object>

In [6]:
# What is the number of responses for each question
q4 = """
SELECT 
    question,
    COUNT(*) as total_responses
FROM survey
GROUP BY question
ORDER BY question;
"""
res_result = show_result(q4)
res_result

,question,total_responses
0,1. What are you looking for?,500
1,2. What's your fit?,475
2,3. Which shapes do you like?,380
3,4. Which colors do you like?,361
4,5. When was your last eye exam?,270


<IPython.core.display.Javascript object>

In [9]:
# Which question(s) of the quiz have a lower completion rates?
q5 = """
SELECT
    question,
    total_responses,
    (total_responses / 500.0)*100 as completion_rate
FROM (
SELECT 
    question,
    COUNT(*) as total_responses
FROM survey
GROUP BY question
ORDER BY question
) t1;
"""
res_result = show_result(q5)
res_result

,question,total_responses,completion_rate
0,1. What are you looking for?,500,100.0
1,2. What's your fit?,475,95.0
2,3. Which shapes do you like?,380,76.0
3,4. Which colors do you like?,361,72.2
4,5. When was your last eye exam?,270,54.0


<IPython.core.display.Javascript object>

In [24]:
fig = go.Figure()

fig.add_trace(
    go.Funnel(
        name="questions",
        y=res_result["question"],
        x=res_result["total_responses"],
        textinfo="value+percent initial",
        textposition="inside",
        marker={
            "color": "mediumseagreen"
        },  # ["deepskyblue", "lightsalmon", "tan", "teal", "silver"]
    )
)
fig.update_layout(title="Completion Rate")
fig.show()

<IPython.core.display.Javascript object>

By the time, the users reached to the last question, `almost 50% of the users abandoned` the survey. The highest drop-off is from `2nd to the 3rd question` and from `4th to the last question`.

## Home-Try-On Funnel

Warby Parker’s purchase funnel is:

1. Take the Style Quiz
2. Home Try-On
3. Purchase the Perfect Pair of Glasses

During the Home Try-On stage, they conducted an A/B Test:

1. 50% of the users will get 3 pairs to try on
2. 50% of the users will get 5 pairs to try on

our job is to find out whether or not users who get more pairs to try on at home will be more likely to make a purchase.

In [25]:
# first five rows of quiz table
q6 = """
SELECT *
FROM quiz
LIMIT 5;
"""
show_result(q6)

,user_id,style,fit,shape,color
0,4e8118dc-bb3d-49bf-85fc-cca8d83232ac,Women's Styles,Medium,Rectangular,Tortoise
1,291f1cca-e507-48be-b063-002b14906468,Women's Styles,Narrow,Round,Black
2,75122300-0736-4087-b6d8-c0c5373a1a04,Women's Styles,Wide,Rectangular,Two-Tone
3,75bc6ebd-40cd-4e1d-a301-27ddd93b12e2,Women's Styles,Narrow,Square,Two-Tone
4,ce965c4d-7a2b-4db6-9847-601747fa7812,Women's Styles,Wide,Rectangular,Black


<IPython.core.display.Javascript object>

In [26]:
# first five rows of home_try_on table
q7 = """
SELECT *
FROM home_try_on
LIMIT 5;
"""
show_result(q7)

,user_id,number_of_pairs,address
0,d8addd87-3217-4429-9a01-d56d68111da7,5 pairs,145 New York 9a
1,f52b07c8-abe4-4f4a-9d39-ba9fc9a184cc,5 pairs,383 Madison Ave
2,8ba0d2d5-1a31-403e-9fa5-79540f8477f9,5 pairs,287 Pell St
3,4e71850e-8bbf-4e6b-accc-49a7bb46c586,3 pairs,347 Madison Square N
4,3bc8f97f-2336-4dab-bd86-e391609dab97,5 pairs,182 Cornelia St


<IPython.core.display.Javascript object>

In [27]:
# first five rows of purchase table
q8 = """
SELECT *
FROM purchase
LIMIT 5;
"""
show_result(q8)

,user_id,product_id,style,model_name,color,price
0,00a9dd17-36c8-430c-9d76-df49d4197dcf,8,Women's Styles,Lucy,Jet Black,150
1,00e15fe0-c86f-4818-9c63-3422211baa97,7,Women's Styles,Lucy,Elderflower Crystal,150
2,017506f7-aba1-4b9d-8b7b-f4426e71b8ca,4,Men's Styles,Dawes,Jet Black,150
3,0176bfb3-9c51-4b1c-b593-87edab3c54cb,10,Women's Styles,Eugene Narrow,Rosewood Tortoise,95
4,01fdf106-f73c-4d3f-a036-2f3e2ab1ce06,8,Women's Styles,Lucy,Jet Black,150


<IPython.core.display.Javascript object>

In [32]:
q9 = """
SELECT DISTINCT q.user_id,
    h.user_id IS NOT NULL as "is_home_try_on",
    h.number_of_pairs,
    p.user_id IS NOT NULL as "is_purchase"
FROM quiz as q
LEFT JOIN home_try_on as h
    ON q.user_id = h.user_id
LEFT JOIN purchase as p
    ON p.user_id = q.user_id
LIMIT 10;
"""
show_result(q9)

,user_id,is_home_try_on,number_of_pairs,is_purchase
0,bbb02cf2-1dfd-4c72-9292-837dba555414,True,5 pairs,False
1,655b9f0c-64b4-4d04-8b81-4a5223ace1d2,True,5 pairs,False
2,af7afcf3-5bfd-4512-80c1-285cd8a23cef,True,3 pairs,False
3,6d6e0935-457d-4ca5-be23-9deb5a39df44,True,3 pairs,True
4,5a3ee321-517d-4a21-a351-d6815ab2edd5,True,5 pairs,True
5,858b658f-5121-45a1-92d4-1a17b3ab3797,False,None,False
6,24ffbce6-5652-4c41-9dab-8b3d4555b8be,True,5 pairs,True
7,b5e4525f-a92e-4b8b-a63c-fa71279621e5,True,3 pairs,True
8,d004787c-07e8-4e4b-a3b0-470e466a1a82,False,None,False
9,0bd5ab3d-a4c9-4cdb-a2ef-ef3d2c5e2db0,True,5 pairs,True


<IPython.core.display.Javascript object>

In [35]:
q10 = """

WITH q AS(
SELECT 
    '1-quiz' as stage,
    COUNT(DISTINCT user_id) as total_count
FROM quiz
),
h AS(
SELECT 
    '2-home_try_on' as stage,
    COUNT(DISTINCT user_id) as total_count
FROM home_try_on
),
p AS(
SELECT 
    '3-purchase' as stage,
    COUNT(DISTINCT user_id) as total_count
FROM purchase
)

SELECT *
FROM q

UNION ALL 

SELECT *
FROM h

UNION ALL 

SELECT *
FROM p;
"""
ht_funnel = show_result(q10)
ht_funnel

,stage,total_count
0,1-quiz,1000
1,2-home_try_on,750
2,3-purchase,495


<IPython.core.display.Javascript object>

In [36]:
fig = go.Figure()

fig.add_trace(
    go.Funnel(
        y=ht_funnel["stage"],
        x=ht_funnel["total_count"],
        textinfo="value+percent initial",
        textposition="inside",
        marker={
            "color": "mediumseagreen"
        },  # ["deepskyblue", "lightsalmon", "tan", "teal", "silver"]
    )
)
fig.update_layout(title="Home-Try-On Funnel")
fig.show()

<IPython.core.display.Javascript object>

Out of all the people who started the quiz, `75%` of them fill out their address details for 'home_try_on' and only `50% `of the people end up `buying` on warby parker.

## A/B Test

In [40]:
q11 = """
WITH result AS(
SELECT DISTINCT q.user_id,
    h.user_id IS NOT NULL as "is_home_try_on",
    h.number_of_pairs,
    p.user_id IS NOT NULL as "is_purchase"
FROM quiz as q
LEFT JOIN home_try_on as h
    ON q.user_id = h.user_id
LEFT JOIN purchase as p
    ON p.user_id = q.user_id
)

SELECT 
    number_of_pairs as AB_Variant,
    SUM(is_home_try_on::INTEGER) as home_trial,
    SUM(is_purchase::INTEGER) as purchase
FROM result
WHERE number_of_pairs IS NOT NULL
GROUP BY 1
"""
ab_test_result = show_result(q11)
ab_test_result

,ab_variant,home_trial,purchase
0,3 pairs,379,201
1,5 pairs,371,294


<IPython.core.display.Javascript object>

In [48]:
ab_test_result["conversion_rate"] = round(
    ab_test_result.apply(lambda x: x["purchase"] / x["home_trial"], axis="columns"),
    2,
)
ab_test_result

,ab_variant,home_trial,purchase,conversion_rate
0,3 pairs,379,201,0.53
1,5 pairs,371,294,0.79


<IPython.core.display.Javascript object>

We can see that the group which receives `5 pairs of glasses` has a `higher conversion rate` than the group which receives `3 pairs`.